<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/1-poor-performances-on-test-set/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!databricks configure --host https://community.cloud.databricks.com/
!pip install -q mlflow

Username: gfesta24@gmail.com
Password: 
Repeat for confirmation: 


In [2]:
%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
import mlflow
from sklearn.model_selection import ParameterGrid
TRAIN_PATH = "train"
TEST_PATH ="test"


In [8]:
import mlflow
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/gfesta24@gmail.com/2DFERResNet")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/4292817106382685', creation_time=1685048826432, experiment_id='4292817106382685', last_update_time=1685634027802, lifecycle_stage='active', name='/Users/gfesta24@gmail.com/2DFERResNet', tags={'mlflow.experiment.sourceName': '/Users/gfesta24@gmail.com/2DFERResNet',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'gfesta24@gmail.com',
 'mlflow.ownerId': '1923923806180228'}>

In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

batch_size = 1024 

In [9]:
import zipfile


zip_ref = zipfile.ZipFile('fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

In [11]:
import os
class_dir=os.listdir(TRAIN_PATH+'/')
folder_names = class_dir
label_dict = {folder_names[i]:i for i in range(len(folder_names))}
num_classes=len(label_dict)
train_image_filenames = []
train_labels = []

# iterate through each folder and collect filenames and labels
for folder_name in folder_names:
    folder_path = os.path.join(TRAIN_PATH, folder_name)
    for filename in os.listdir(folder_path):
        train_image_filenames.append(os.path.join(TRAIN_PATH+"/"+folder_name, filename))
        train_labels.append(label_dict[folder_name])

# create pandas dataframe
train_df = pd.DataFrame({'filename': train_image_filenames, 'emotion': train_labels})

print(len(train_df))


test_image_filenames = []
test_labels = []
class_dir=os.listdir(TEST_PATH+'/')
for folder_name in folder_names:
    folder_path = os.path.join(TEST_PATH, folder_name)
    for filename in os.listdir(folder_path):
        test_image_filenames.append(os.path.join(TEST_PATH+"/"+folder_name, filename))
        test_labels.append(label_dict[folder_name])

test_df = pd.DataFrame({'filename': test_image_filenames, 'emotion': test_labels})

28709


In [12]:
train_df['img_as_matrix'] = train_df['filename'].apply(lambda path: cv2.imread(path))
test_df['img_as_matrix'] = test_df['filename'].apply(lambda path: cv2.imread(path))


In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data.iloc[index]['img_as_matrix'] 
        label = self.data.iloc[index]['emotion'] 

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [14]:
train_df.drop(columns=["filename"], inplace=True)
test_df.drop(columns=["filename"], inplace=True)
train_df.at[0,"img_as_matrix"]
full_df = pd.concat([train_df, test_df])

full_dataset = CustomDataset(full_df, transform=transforms.ToTensor())
full_daset_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=False)



#train_dataset = CustomDataset(train_df, transform=transforms.ToTensor())
#test_dataset = CustomDataset(test_df, transform=transforms.ToTensor())
 

#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X_train, X_test, y_train, y_test = train_test_split(full_df['img_as_matrix'], full_df['emotion'], test_size=0.33, random_state=42, stratify=full_df['emotion'])

print(type(X_train))
def mean_std_calc(loader):
  cnt = 0
  fst_moment = torch.empty(3)
  snd_moment = torch.empty(3)

  for images, _ in loader:
      b, c, h, w = images.shape
      nb_pixels = b * h * w
      sum_ = torch.sum(images, dim=[0, 2, 3])
      sum_of_square = torch.sum(images ** 2,
                                dim=[0, 2, 3])
      fst_moment = (cnt * fst_moment + sum_) / (
                    cnt + nb_pixels)
      snd_moment = (cnt * snd_moment + sum_of_square) / (
                          cnt + nb_pixels)
      cnt += nb_pixels

  mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)
  return mean, std

#Normalizzazione train loader
mean, std = mean_std_calc(full_daset_loader)

transform_img_normal = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std= std),
])

X_train.name="img_as_matrix"
X_test.name="img_as_matrix"
y_test.name="emotion"
y_train.name="emotion"
new_train_df=pd.concat([X_train, y_train], axis=1)
new_test_df=pd.concat([X_test, y_test], axis=1)
print(new_train_df)

train_dataset = CustomDataset(new_train_df, transform=transform_img_normal)

test_dataset = CustomDataset(new_test_df, transform=transform_img_normal)




<class 'pandas.core.series.Series'>
                                           img_as_matrix  emotion
625    [[[48, 48, 48], [52, 52, 52], [54, 54, 54], [5...        0
1345   [[[38, 38, 38], [45, 45, 45], [48, 48, 48], [5...        0
9439   [[[144, 144, 144], [137, 137, 137], [134, 134,...        1
6114   [[[108, 108, 108], [24, 24, 24], [0, 0, 0], [1...        6
12430  [[[3, 3, 3], [1, 1, 1], [2, 2, 2], [4, 4, 4], ...        2
...                                                  ...      ...
23339  [[[253, 253, 253], [248, 248, 248], [252, 252,...        5
21635  [[[182, 182, 182], [172, 172, 172], [154, 154,...        5
14447  [[[63, 63, 63], [64, 64, 64], [58, 58, 58], [5...        2
14240  [[[181, 181, 181], [175, 175, 175], [161, 161,...        2
17100  [[[175, 175, 175], [148, 148, 148], [101, 101,...        3

[24044 rows x 2 columns]


In [15]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;

# Resnet 50 Pytorch Code

In [16]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super().__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels,
            intermediate_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * 4, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)        
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.fc2(x)
        x = self.bn3(x)
        x = self.fc3(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channel=3, num_classes=7):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


In [36]:
mlflow.end_run()

In [ ]:

param_grid = {'batch_size': [64, 128, 256, 512],'lr': [0.01, 0.001, 0.0001], "momentum":[0.7, 0.8, 0.9], "decay":[0.001, 0.01, 0.1]}
expanded_grid = ParameterGrid(param_grid)


for i in range(len(expanded_grid)):
  best_loss=100
  best_model_train_acc=0
  best_model_test_acc=0
  best_model_test_loss=0
  best_model_train_loss=0
  min_delta=0
  patience=3
  train_loader = DataLoader(train_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=True)
  
  test_loader = DataLoader(test_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=False)
  model = ResNet50().to(device)

  criterion = nn.CrossEntropyLoss()

  optimizer = torch.optim.SGD(model.parameters(), lr=expanded_grid[i]['lr'], momentum =expanded_grid[i]['momentum'])


  mlflow.start_run()


  n_total_steps = len(train_loader)
  num_epochs = 100
  acc_list_train=[]
  acc_list_test=[]
  model.train()


  mlflow.set_tag("model_name", "ResNet50")
  mlflow.log_param("lr", expanded_grid[i]['lr'])
  mlflow.log_param("momentum", expanded_grid[i]['momentum'])
  mlflow.log_param("batch_size", expanded_grid[i]['batch_size'])
  mlflow.log_param("decay", expanded_grid[i]['decay'])
  best_loss = 100
  counter=0
  stop=False

  for epoch in range(num_epochs):
      print(counter)
      if stop:
        print(stop)
        break
      running_loss = 0.0
      running_acc = 0.0
      seen = 0
      for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)
        
        _, acc = accuracy(outputs, labels)
        seen +=labels.shape[0]

        optimizer.zero_grad()      
        loss.backward()            
        optimizer.step()  
        running_loss += loss.item()    
        running_acc += acc

      print (f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {running_acc/seen:.4f}')
      acc_list_train.append(running_acc/len(train_loader))
      mlflow.log_metric("train_loss", running_loss / len(train_loader), step=epoch)
      mlflow.log_metric("train_acc", running_acc/seen, step=epoch)
      

      tot_corrette = 0
      tot_eseguite = 0
      running_test_loss = 0
      val_loss = 0
      
      with torch.no_grad():

        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
          
            outputs = model(images)
            test_loss = criterion(outputs, labels)
            _, n_corrette=accuracy(outputs, labels)
            
            running_test_loss += test_loss.item() 
            tot_corrette+=n_corrette.item()
            tot_eseguite+=labels.shape[0]

        test_acc=100* (tot_corrette/tot_eseguite)
        val_loss = running_test_loss / len(test_loader)
        acc_list_test.append(test_acc)
        print("Test acc: ", test_acc)
        print("Test loss: ", val_loss)
        
        mlflow.log_metric("test_acc", test_acc, step=epoch)
        mlflow.log_metric("test_loss", val_loss, step=epoch)
        
        
      if val_loss < best_loss - min_delta:
        print("MIGLIORATO")
        best_loss = val_loss
        best_model_train_acc=running_acc/seen
        best_model_test_acc=test_acc
        best_model_test_loss=val_loss
        best_model_train_loss=running_loss / len(train_loader)
        counter = 0
        # Salva i pesi del modello se la validation loss è migliorata
        torch.save(model.state_dict(), 'best_model.pt')
      else:
        counter += 1
      # Verifica se raggiunto il criterio di early stopping
        if counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            stop=True
      print("BEST TEST LOSS: ", best_loss)

  mlflow.set_tag("Epochs_stopped", epoch+1)
  mlflow.log_artifact("best_model.pt")
  mlflow.log_metric("best_test_acc", best_model_test_acc)
  mlflow.log_metric("best_test_loss", best_model_test_loss)
  mlflow.log_metric("best_train_acc", best_model_train_acc)
  mlflow.log_metric("best_train_loss", best_model_train_loss)
  mlflow.end_run()

  print('Finished Training')

  PATH = './cnn.pth'
  torch.save(model.state_dict(), PATH)
  torch.cuda.empty_cache()

In [ ]:

# Creazione dell'asse x con gli indici delle liste
epochs = range(1, len(acc_list_train) + 1)

# Tracciamento delle due variabili come linee di colori diversi
plt.plot(epochs, acc_list_train, 'b', label='Training Accuracy')
plt.plot(epochs, acc_list_test, 'r', label='Test Accuracy')

# Titoli degli assi
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

# Aggiunta di una legenda
plt.legend()

# Visualizzazione del grafico
plt.show()

In [ ]:
with torch.no_grad():
    tot_corrette = 0
    tot_eseguite = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
       
        outputs = model(images)
        _, n_corrette=accuracy(outputs, labels)
        tot_corrette+=n_corrette
        tot_eseguite+=labels.shape[0]